In [ ]:
import scanpy as sc
import pandas as pd
import scvelo as scv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn


In [ ]:
crypto_3 = sc.read('../../../Data/notebooks_data/crypto_3.h5ad')

In [ ]:
MT = ['MT' in i for i in crypto_3.var_names]
perc_mito = np.sum( crypto_3[:,MT].X, 1 ) / np.sum( crypto_3.X, 1 )
crypto_3.obs['perc_mito'] = perc_mito.copy()

In [ ]:
sc.pl.scatter(crypto_3, x='total_counts', y='n_genes_by_counts', color='perc_mito', 
              title ='Nr of transcripts vs Nr detected genes, coloured by mitocondrial content')

In [ ]:
sns.distplot(crypto_3.obs['total_counts'], bins=50)

In [ ]:
sns.distplot(crypto_3.obs['n_genes_by_counts'], bins=50)

In [ ]:
MIN_COUNTS = 3000
MAX_COUNTS = 15000
MIN_GENES = 2500
MAX_GENES = 5000
MAX_MITO = .1

In [ ]:
sc.pl.scatter(crypto_3[ crypto_3.obs['total_counts']<MAX_COUNTS ], 
              x='total_counts', y='n_genes_by_counts', color='perc_mito',
              title ='Nr of transcripts vs Nr detected genes, coloured by mitocondrial content')

In [ ]:
sc.pl.scatter(crypto_3[ crypto_3.obs['n_genes_by_counts'] > MIN_GENES ], 
              x='total_counts', y='n_genes_by_counts', color='perc_mito',
              title ='Nr of transcripts vs Nr detected genes, coloured by mitocondrial content')

In [ ]:
sc.preprocessing.filter_cells(crypto_3, max_counts=MAX_COUNTS)

sc.preprocessing.filter_cells(crypto_3, min_counts=MIN_COUNTS)

sc.preprocessing.filter_cells(crypto_3, min_genes=MIN_GENES)

sc.preprocessing.filter_cells(crypto_3, max_genes=MAX_GENES)

sc.preprocessing.filter_genes(crypto_3, min_cells=10)

crypto_3 = crypto_3[crypto_3.obs['perc_mito']<MAX_MITO].copy()

In [ ]:
crypto_3

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.ensemble import IsolationForest

df = crypto_3.obs[ ['n_genes_by_counts', 'total_counts', 
                    'perc_mito', 'pct_counts_in_top_50_genes'] ]

df2 = scale(df, axis=0)

pca = PCA(n_components=2)

Y = pca.fit_transform(df2)

clf = IsolationForest(random_state=0, contamination=.1)

pred = clf.fit_predict(df2)

pred = pd.Categorical(pred)
pred = pred.rename_categories(['Outlier','Cell'])
df['Category'] = pred

sns.scatterplot(Y[:,0],Y[:,1], hue = df.total_counts, 
                size=df.n_genes_by_counts, style = df.Category, 
                palette="Blues", sizes=(20, 200), hue_norm=(0, 100))

In [ ]:
pred

In [ ]:
crypto_3 = crypto_3[pred=='Cell'].copy()

In [ ]:
crypto_3

In [ ]:
sc.external.pp.scrublet(crypto_3, expected_doublet_rate=0.06, threshold=.2, random_state=12345)

In [ ]:
sns.distplot(crypto_3.obs['doublet_score'])

In [ ]:
crypto_3 = crypto_3[np.invert(crypto_3.obs['predicted_doublet'])].copy()

In [ ]:
# TPM normalization and matrix copy
sc.pp.normalize_per_cell(crypto_3)
crypto_3.layers['umi_tpm'] = crypto_3.X.copy()

# matrix logarithmization and copy
sc.pp.log1p(crypto_3)
crypto_3.layers['umi_log'] = crypto_3.X.copy()

sc.pp.highly_variable_genes(crypto_3, n_top_genes=15000)

crypto_3.write('../../../Data/notebooks_data/crypto_3.filt.h5ad')

# standardized gene expressions and matrix copy
sc.pp.scale(crypto_3)
crypto_3.layers['umi_gauss'] = crypto_3.X.copy()

sc.preprocessing.pca(crypto_3, svd_solver='arpack', random_state=12345)

crypto_3

sc.plotting.pca_variance_ratio(crypto_3)

In [ ]:
plt.rcParams['figure.figsize'] = (6,6) #reduce figure size
sc.pl.pca(crypto_3, color=['total_counts','SYCP1'])

sc.pp.neighbors(crypto_3, n_pcs=15)

sc.tools.umap(crypto_3, random_state=54321, n_components=2)

sc.plotting.umap(crypto_3, color=['total_counts'])

In [ ]:
sc.plotting.umap(crypto_3, color=['perc_mito'])